In [ ]:
#default_exp navigator

In [ ]:
#hide
#test_flag_colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#hide
!pip install nbdev
!pip install fastcore

In [ ]:
#hide
%cd /content/drive/My\ Drive/fa_convnav

In [ ]:
#hide
# not deps but we need them to use nbdev and run tests
from nbdev import * 
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#hide
try:
  import fastai2.basics
except:
  !pip install fastai2
else:
  print('fastai2 already installed')

In [ ]:
#hide
from fastai2.basics import *
from fastai2.callback.all import *
from fastai2.vision.all import *
from torch import torch

pets = DataBlock(blocks=(ImageBlock, CategoryBlock), 
                 get_items=get_image_files, 
                 splitter=RandomSplitter(),
                 get_y=RegexLabeller(pat = r'/([^/]+)_\d+.jpg$'),
                 item_tfms=Resize(460),
                 batch_tfms=[*aug_transforms(size=224, max_rotate=30, min_scale=0.75), Normalize.from_stats(*imagenet_stats)])

dls = pets.dataloaders(untar_data(URLs.PETS)/"images",  bs=128)

model = resnet18
learn = cnn_learner(
    dls, 
    model, 
    opt_func=partial(Adam, lr=slice(3e-3), wd=0.01, eps=1e-8), 
    metrics=error_rate, 
    config=cnn_config(ps=0.33)).to_fp16()

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth


In [ ]:
#hide
import gzip
def get_test_vars():
  #load test_vars from file if not already downloaded
  try:
    test_learner
  except:
    with gzip.open("test_learner_resnet18", "rb") as f:
      test_learner = pickle.load(f)
    with gzip.open("test_summary_resnet18", "rb") as f:
      test_summary = pickle.load(f)
  try:
    test_df
  except:
    with gzip.open("test_df_resnet18", "rb") as f:
      test_df = pickle.load(f)
  return test_learner, test_summary, test_df

# Navigator

> CNN viewer and navigator

In [ ]:
#export
import pickle
from fa_convnav.models import models
from fa_convnav.core import *
from pandas import DataFrame, option_context, concat
from math import ceil

In [ ]:
#export
def convnav_supported_models():
  "Prints list of transfer learning models supported by fa_convnav"
  supported_models()

In [ ]:
#export
def cndf_view(df, verbose=3, truncate=0, tight=True, align_cols='left', top=False):
    "Display a valid CNDF dataframe `df` with optional arguments and styling"

    def check_view_args(df, verbose, truncate):
      assert type(df) == DataFrame and 'Module_name' in df.columns, "Not a valid convnav dataframe"
      assert isinstance(truncate, int) and -10 <= truncate <= 10, f"Argument 'truncate' must be an integer between -10 (show more cols) and +10 (show fewer cols)"
      assert isinstance(verbose, int) and 1 <= verbose <= 5, f"Argument verbose must be 1 2 or 3 "

    def display_df(df, verbose, truncate, tight, align_cols):
      with option_context("display.max_rows", 1000):
        df.index.name = 'Index' 
        df_styled = df.iloc[:,:-(11+truncate)].style.set_properties(**{'text-align': align_cols})
        if tight: 
          display(df_styled)
        else:
          display(df.iloc[:,:-(11+truncate)]) 

    #handle arguments
    check_view_args(df, verbose, truncate)
    if not isinstance(tight, bool): tight = True
    if len(df) < 10: tight = False
    if verbose != 3: truncate = (10, 4, 0, 0, -10)[verbose-1]
    if verbose == 4: df = add_container_row_info(df)

    #display df
    if top and len(df) > 10:
      display_df(df.iloc[:10], verbose, truncate, False, align_cols)
      print(f'...{len(df)-10} more layers')
    elif len(df) > 0: 
      display_df(df, verbose, truncate, tight, align_cols)
    else:
      print('No data to display')
    return None


*   `verbose`: 1 = Index and Layer_name columns only; 2 = Model structure; 3 = Model Structure and layer_info (output dims, params and frozen/unfrozen);  4 = include layer_info and block/layer counts in container rows; 5 = expose hidden columns.
*   `tight`: True = tight layout with minimal space between rows (best for large models with many rows to display). False = display dataframe with normal row spacing. The default is True but less than 10 rows are automatically displayed with normal spacing.
*   `truncate`: truncate number displayed columns by integer value between -10 and 10. 0 = default. Negative values reveal hidden columns. Overidden when the verbose argument is set to a non-default setting.
*   `align_cols`: 'left' or 'right' alignment of column data
*   `top`: display first 10 rows only followed by a count of undisplayed rows 


In [ ]:
#export
def add_container_row_info(df):
    "Add output dimensions and block/layer counts to container rows of `df`. These are not added when a CNDF dataframe is first built to avoid cluttering the display of larger dataframes."
    df.loc[df['Division'] == '', 'Division'] = df['div_id']
    df.loc[df['Container_child'] == '', 'Container_child'] = df['chd_id']
    df.loc[df['Container_block'] == '', 'Container_block'] = df['blk_id']
    df.loc[df['Output_dimensions'] == '', 'Output_dimensions'] = df['out_dim']
    df.loc[df['Currently'] == '', 'Currently'] = df['current']
    return df

In [ ]:
#export
def cndf_search(df, searchterm, exact=True, show=True):
  "Search a CNDF dataframe, display the results in a dataframe and return matching module object(s)"

  def match(df, searchterm, exact):
    "Searches `df` for `searchterm`, returning exact matches only if `exact=True` otherwise any match"

    #select 'df' row using index from 'searchterm'
    if isinstance(searchterm, int):
      assert searchterm >= 0 and searchterm <= len(df), f'Layer ID out of range: min 0, max {len(df)}'
      x = df.iloc[searchterm].copy()
      x = DataFrame(x).transpose()
      return x    
      
    #if searchterm is a float assume it is a layer name (i.e. format 0.0.1) and convert to string
    if isinstance(searchterm, float): searchterm = str(searchterm)

    #select rows matching the conditional df[key] ==/contains value (exact=True/false) for dict
    if isinstance(searchterm, dict):
      
      for col, s in searchterm.items():
        assert col in df.columns, f'{col} not a valid column identifier. Valid column names are {df.columns}'
        return df[df[col] == s].copy() if exact else df[df[col].str.contains(s)].copy()
      return x

    #select rows in df where df[col] ==/contains searchterm string (exact=True/False) 
    if isinstance(searchterm, str):
      searchterm = searchterm.strip(' \.')
      cols = {'Module_name', 'Torch_class', 'Division', 'Container_child', 'Container_block', 'Layer_description'}
      if exact: 
        for col in cols:
          x = df[df[col] == searchterm].copy()
          if not x.empty: return x
      else: 
        for col in cols:
          x = df[df[col].str.contains(searchterm)].copy()
          if not x.empty: return x
      return x
         
    assert True, 'Unrecognizable searchterm'
        
  def process_searchterm(df, searchterm, exact): 
    "Search 'df` for single or combination of modules and layers. If df = None, searches instance dataframe `self._cndf` (default)"

    #if searchterm is a `float` assume it is a layer name (i.e. format 0.0.1) and convert to `string`
    if isinstance(searchterm, float): searchterm = str(searchterm)

    if isinstance(searchterm, int): 
      return match(df, searchterm, True)
    elif isinstance(searchterm, str): 
      return match(df, searchterm, exact)
    #concatenate successive search results (logical 'OR') for series of dicts
    elif isinstance(searchterm, dict):
      _df = DataFrame()  
      for col, s in searchterm.items():
        new_df = match(df, {col:s}, exact)
        _df = concat((_df, new_df), axis=0, ignore_index=False).drop_duplicates('Module_name')
      return _df
    #concatenate successive search results (logical 'OR') in list
    elif isinstance(searchterm, list):
      _df = DataFrame()  
      for s in searchterm: 
        new_df = match(df, s, exact)
        _df = concat((_df, new_df), axis=0, ignore_index=False).drop_duplicates('Module_name')
      return _df
    #recursively call match on _df to logical 'AND' successive search results in tuple
    elif isinstance(searchterm, tuple):
      _df = df.copy()
      for s in searchterm:
        _df = match(_df, s, exact)
      return _df
    else: 
      assert True, 'Unrecognizable searchterm'

  #get search results
  df_res = process_searchterm(df, searchterm, exact)

  #show matches and return corresponding modules
  if df_res is not None and not df_res.empty:
    if show:
      print(f'{len(df_res)} layers found matching searchterm(s): {searchterm}\n')
      cndf_view(df=df_res)
    return df_res['lyr_obj'].tolist()
  else: 
    if show: print(f'No matches for searchterm(s): {searchterm}\n')
    return None

`Searchterm` can be:

*   `int` : the module with Index number `int` is returned
*   `float`: module(s) where `str(float)` matches the Layer_name are returned
*   `str`: module(s) with `str` in one of 'Layer_name', 'Torch_class', 'Division', 'Module', 'Block', 'Layer_description' are returned. Columns are searched in this order with the search ending with the first column to make a match/matches. 
*   `dict`, e.g. {'col', 'str'} matches `str` in column `col` 

Searchterms can also be a combined as follows:

*   `[101, 102, 105]` logical OR of rows matching indexes `101`, `102` plus `103`
*   `('0.5', 'conv2d')` logical AND of rows matching `0.5` in Layer_name and `conv2d` in `Layer_description`
*   `{{'col1', 'str1'}, {'col2', 'str2'}}` logical OR of matches `str1` in `col1` plus `str2` in `col`'.

Return only exact matches between searchterm and column entry with exact = True (default).
<br />



.

In [ ]:
%%capture
_, _, test_df = get_test_vars()
test_df['lyr_obj'] = None

test_eq(len(cndf_search(test_df, 12, )), 1)
test_eq(len(cndf_search(test_df, '0.6.1.conv2')), 1)
test_eq(len(cndf_search(test_df, 0.6, exact=False)), 16)
test_eq(len(cndf_search(test_df, {'Module_name': '0.6', 'Layer_description':'Conv2d'}, exact=True)), 1)
test_eq(len(cndf_search(test_df, ['0.6', '0.5'], exact=False)), 32)
test_eq(cndf_search(test_df, ('0.6', '0.5'), exact=False), None)

In [ ]:
#export
class ConvNav(CNDF):
  "Builds a CNDF dataframe representation of a CNN model from a fastai `Learner` object and `Learner.summary()`. \
  Provides methods to view, search and select model layers and modules for further investigation."

  def __init__(self, learner, learner_summary):
    super().__init__(learner, learner_summary)

  def __len__(self):
    return len(self._cndf)

  def __str__(self) -> str:
    return self.model_info

  def __call__(self):
    self.view(top=True)

  def __contains__(self, s):
    return self.search(s)

  def search(self, searchterm, **kwargs):
    "Find `searchterm` in instance dataframe, display the results and return matching module object(s). See `cndf_search()` for kwargs."
    return cndf_search(self._cndf, searchterm, **kwargs)

  def view(self, **kwargs):
    "Display instance CNDF dataframe with optional arguments and styling (see `cndf_view()` for kwargs)"
    print(f'{self.model_info}\n')
    cndf_view(self._cndf, **kwargs)

  def _view(self, df, add_info=False, **kwargs):
    "Add output dimensions and block/layer counts to Container rows of `df` then display `df`"
    _df = df.copy()
    if add_info: 
      _df = add_container_row_info(_df)
    cndf_view(_df, **kwargs)

  @property
  def head(self):
    "View module of model head"
    df = self._cndf.copy()
    df = df[df['Module_name'].str.startswith('1')]
    if not df.empty:
      res = f"{self.model_type.capitalize()}: {self.model_name.capitalize()}\n"
      res += f"Input shape: {self._cndf.iloc[1]['out_dim']} (bs, filt, h, w)\n"
      res += f"Output features: {self.output_dimensions} (bs, classes)\n" 
      print(res)
      self._view(df, truncate=1)
    else:
      res = "Model has no head"
      print(res)


  @property
  def body(self):
    "View modules of model body"
    df = self._cndf.copy()
    df = df.loc[df['Module_name'].str.startswith('0')]
    if not df.empty:
      res = f"{self.model_type.capitalize()}: {self.model_name.capitalize()}\n"
      res += f"Input shape: {self.input_sizes} (bs, ch, h, w)\n"
      res += f"Output dimensions: {df.iloc[-1]['Output_dimensions']} (bs, filt, h, w)\n"
      res += f"Currently frozen to parameter group {self.frozen_to} out of {self.num_param_groups}\n" 
      print(res)
      self._view(df)
    else:
      res = "Model body has no contents"
      print(res)


  @property
  def divs(self):
    "Summary info for model head and body"
    df = self._cndf.copy()
    df = df.loc[(df['Module_name'] == '0') | (df['Module_name'] == '1')]

    for i in range(2):
      df_div = self._cndf.loc[self._cndf['div_id'] == str(i)].copy()
      df.iloc[i]['Model'] = self.model_name
      df.iloc[i]['Container_child'] = len(df_div[df_div['Container_child'] != ''])
      df.iloc[i]['Container_block'] = len(df_div[df_div['Container_block'] != ''])
      df.iloc[i]['Layer_description'] = len(df_div[df_div['Layer_description'] != ''])
      params = df_div['Parameters'].values
      params_int = [int(x.replace(',','')) for x in params if x != '']
      params_summed = sum(params_int)
      df.iloc[i]['Parameters'] = params_summed

    df['Output_dimensions'] = df['out_dim']
    df.iloc[0]['Currently'] = df.iloc[0]['current']

    df = df.rename(columns={'Container_child': 'Container_child (num)', 'Container_block': 'Container_block (num)', 'Layer_description': 'Layers'})
    res = f"{self.model_type.capitalize()}: {self.model_name.capitalize()}\n"
    res += f"Input shape: [{self.inp_sz}] (bs, ch, h, w)\n"
    res += f"Divisions:  body (0), head (1)\n"
    print(res)
    self._view(df)


  @property
  def dim_transitions(self):
    "Finds layers with different input and output dimensions. These are useful points to apply hooks and callbacks for investigating model activity."
    df = self._cndf.copy()
    df = df[df['Torch_class'].str.contains('Conv2d')]

    n = []
    old_dims = 0
    for i, row in enumerate(df.iterrows()):
      row=row[1]
      new_dims = row['Output_dimensions'].rstrip(']').split(' x ')[-1]
      if new_dims != old_dims:
        n.append(i)
        old_dims = new_dims
    df = df.iloc[n]

    print(f"{self.model_name.capitalize()}\nLayer dimension changes\n")
    self._view(df, add_info=True)
    return df['lyr_obj'].tolist()


  @property
  def linear_layers(self):
    "Displays and returns all linear layers in the `model`"
    df = self._cndf.copy()
    df = df[df['Torch_class'].str.contains('Linear')]
    df['Division'] = df['div_id']

    print(f"{self.model_name.capitalize()} linear layers\n")
    self._view(df, truncate=1, tight=False)
    return df['lyr_obj'].tolist()


  def frozen_vs_unfrozen(self, col, req):
    "Displays and returns all frozen vs unfrozen modules and layers"
    df = self._cndf.copy()
    df = df.loc[df['div_id'] == '0']
    col = col.lower()
    req_substr = req[req.find(' ')+1:].capitalize()

    if col == 'layer':
      df = df[df['Currently'] == req_substr]
    elif col == 'block':
      df = df.loc[(df['Container_block'] != '') & (df['current'] == req_substr)]
    else:
      df = df.loc[(df['Container_child'] != '') & (df['current'] == req_substr)]

    if req == 'Last frozen': df=df.iloc[-1:, :]
    if req == 'First unfrozen': df=df.iloc[:1, :]

    print(f"{self.model_name}\n{req} {col}(s): ")
    self._view(df, add_info=True)
    return df['lyr_obj'].tolist()

  def frozen(self, col='child'):
    "Displays and returns all frozen child container (`col='child'`), block container (`col='block'`) or layers (`col='layer'`)."
    return self.frozen_vs_unfrozen(col, 'All frozen')

  def unfrozen(self, col='child'):
    "Displays and returns all unfrozen child container (`col='child'`), block container (`col='block'`) or layers (`col='layer'`)."
    return self.frozen_vs_unfrozen(col, 'All unfrozen')

  def last_frozen(self, col='child'):
    "Displays and returns the last frozen child container (`col='child'`), block container (`col='block'`) or layer (`col='layer'`)."
    return self.frozen_vs_unfrozen(col, 'Last frozen')

  def first_unfrozen(self, col='child'):
    "Displays and returns the first unfrozen child container (`col='child'`), block container (`col='block'`) or layer (`col='layer'`)."
    return self.frozen_vs_unfrozen(col, 'First unfrozen')


  def structs(self, col):
    "Display container elements of model body"
    df = self._cndf.copy()
    df = df.loc[df['div_id'] == '0']
    df = df[df[col] != '']

    df = add_container_row_info(df)

    if col == 'Container_block':
      df['Layer_description'] = df['lyr_blk']
      df.rename(columns={'Layer_description': 'Num_layers'}, inplace=True)

    if col == 'Container_child':
      df['Container_block'] = df['blk_chd']
      df.rename(columns={'Container_block': 'Num_blocks'}, inplace=True)
      df.loc[df['Layer_description'] == '', 'Layer_description'] = df['lyr_chd']
      df.rename(columns={'Layer_description': 'Layer_description (or num layers if > 1)'}, inplace=True)
      
    print(f"{self.model_name}\n{'Blocks' if col == 'Container_block' else 'Children'} ")
    self._view(df, tight=False)
    return df['lyr_obj'].tolist()

  @property
  def children(self):
    "Display and return child container modules (equivelent to fastai `Learner.model.children()`)"
    return self.structs('Container_child')

  @property
  def blocks(self):
    "Display and return block container modules"
    return self.structs('Container_block')


  def find_conv(self, req='all', num=1, in_main=False):
    "Finds the first (`req=First`) or last (`req=last`) `num` Conv2d layers in the model body. Set `in_main=True` to return conv layers from the main body of the model only."

    valid_reqs = ['all', 'first', 'last']
    assert req.lower() in valid_reqs, f'Invalid request: req must be one of {valid_reqs}'

    df = self._cndf.copy()
    df = df[df['tch_cls'] == 'Conv2d']
    assert isinstance(num, int) and num > 0, f'Number must be a positive integer'
    if num >= len(df): num = len(df)-1 

    if in_main:
      df = df.iloc[1:]

    if req == 'first':
      df = df.iloc[:num,:]
    elif req == 'last':
      df = df.iloc[num*-1:,:]
    else: pass

    print(f"{self.model_name}\n{req.capitalize()} {'' if req=='all' else num} Conv2d layers\n")
    self._view(df, tight=False)
    return df['lyr_obj'].tolist()


  def find_block(self, b, layers=True, layers_only=False):
    "Finds, displays and returns container blocks by module name `b`"

    #strip preceeding '0.'s from module name so search starts from same point in all models
    while b.startswith('0') or b.startswith('.'):
      b = str(b).strip('0\.')
    b = b.split('.')
    assert len(b) == 2, "Module name does not specify a block element: try something like '6.1' or '0.6.1'"
    
    df = self._cndf.copy()
    df = df[(df['chd_id'] == b[0]) & (df['blk_id'] == b[1])]

    if not layers:
      df = df[df['Layer_description'] == '']
      df['Layer'] = df['lyr_blk']
      df.rename(columns={'Layer_description': 'Num_layers'}, inplace=True)
      layers_only = False

    if layers_only: 
      df = df[df['Layer_description'] != '']

    print(f"{self.model_name}\nBlock 0.{b[0]}.{b[1]}\n")
    self._view(df, add_info=True)
    return df['lyr_obj'].tolist()


  def spread(self, req='conv', num=5):
    "Returns `num` of equally spaced `req` elements over the model. `req` = 'conv', 'block', or 'child'"

    valid_reqs = ['Child', 'Block', 'Layer', 'Conv2d', 'Conv2D', 'Conv']
    req = req.capitalize()
    assert req in valid_reqs, f'Invalid request: req must be one of {valid_reqs}'
    assert isinstance(num, int), 'Number must be an integer' 
    num = max(3, num)

    df = self._cndf.copy()
    df = df[df['div_id'] == '0'][1:]

    if 'Child' in req:
      df = df[df['Container_child'] == 'Sequential']
    elif req == 'Block':
      df = df[df['Container_block'] != '']
    else: 
      df = df[df['tch_cls'] == 'Conv2d']

    num = min(len(df), num)

    df = add_container_row_info(df)

    if len(df) > num: 
      n = list(range(0, len(df), (ceil(len(df)/num))))
      n.pop()
      n.append(len(df)-1)
      df = df.iloc[n]

    if  req == 'Child' or req == 'Block':
      df['Layer_description'] = df['lyr_blk']
      df.rename(columns={'Layer_description': 'Num_layers'}, inplace=True)

    if req == 'Child':
      df['Container_block'] = df['blk_chd']
      df.rename(columns={'Container_block': 'Num_blocks'}, inplace=True)
      df['Num_layers'] = df['lyr_mod']

    print(f"{self.model_name}\nSpread of {req.lower()} where n = {num}\n")
    self._view(df)
    return df['lyr_obj'].tolist()
      

Fastai2 Learner objects are created from a dataloader, model and optimizer as decribed in the fastai documentation https://dev.fast.ai/ (also `03_examples00.ipynb` and `04_examples01.ipynb`). Run `convnav_supported_models()` from a notebook cell to see the list of supported models.


```
cn = ConvNav(Learner, Learner.summary()
```

In [ ]:
test_learner, test_summary, _ = get_test_vars()
cn_test = ConvNav(test_learner, test_summary)

test_eq(type(cn_test._cndf), DataFrame) 
test_eq(len(cn_test._cndf), 79)             # rows
test_eq(len(cn_test._cndf.columns), 22)     # columns   


###Search and view the CNDF dataframe.

In [ ]:
show_doc(ConvNav.search)

<h4 id="ConvNav.search" class="doc_header"><code>ConvNav.search</code><a href="__main__.py#L19" class="source_link" style="float:right">[source]</a></h4>

> <code>ConvNav.search</code>(**`searchterm`**, **\*\*`kwargs`**)

Find `searchterm` in instance dataframe, display the results and return matching module object(s). See `cndf_search()` for kwargs.

In [ ]:
show_doc(ConvNav.view)

<h4 id="ConvNav.view" class="doc_header"><code>ConvNav.view</code><a href="__main__.py#L23" class="source_link" style="float:right">[source]</a></h4>

> <code>ConvNav.view</code>(**\*\*`kwargs`**)

Display instance CNDF dataframe with optional arguments and styling (see `cndf_view()` for kwargs)

### Additional methods:

In [ ]:
show_doc(ConvNav.head)

<h4 id="ConvNav.head" class="doc_header"><code>ConvNav.head</code><a href="" class="source_link" style="float:right">[source]</a></h4>

View module of model head

In [ ]:
show_doc(ConvNav.body)

<h4 id="ConvNav.body" class="doc_header"><code>ConvNav.body</code><a href="" class="source_link" style="float:right">[source]</a></h4>

View modules of model body

In [ ]:
show_doc(ConvNav.divs)

<h4 id="ConvNav.divs" class="doc_header"><code>ConvNav.divs</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Summary info for model head and body

In [ ]:
show_doc(ConvNav.dim_transitions)

<h4 id="ConvNav.dim_transitions" class="doc_header"><code>ConvNav.dim_transitions</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Finds layers with different input and output dimensions. These are useful points to apply hooks and callbacks for investigating model activity.

In [ ]:
show_doc(ConvNav.linear_layers)

<h4 id="ConvNav.linear_layers" class="doc_header"><code>ConvNav.linear_layers</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Displays and returns all linear layers in the `model`

In [ ]:
show_doc(ConvNav.frozen)

<h4 id="ConvNav.frozen" class="doc_header"><code>ConvNav.frozen</code><a href="__main__.py#L150" class="source_link" style="float:right">[source]</a></h4>

> <code>ConvNav.frozen</code>(**`col`**=*`'child'`*)

Displays and returns all frozen child container (`col='child'`), block container (`col='block'`) or layers (`col='layer'`).

In [ ]:
show_doc(ConvNav.unfrozen)

<h4 id="ConvNav.unfrozen" class="doc_header"><code>ConvNav.unfrozen</code><a href="__main__.py#L154" class="source_link" style="float:right">[source]</a></h4>

> <code>ConvNav.unfrozen</code>(**`col`**=*`'child'`*)

Displays and returns all unfrozen child container (`col='child'`), block container (`col='block'`) or layers (`col='layer'`).

In [ ]:
show_doc(ConvNav.last_frozen)

<h4 id="ConvNav.last_frozen" class="doc_header"><code>ConvNav.last_frozen</code><a href="__main__.py#L158" class="source_link" style="float:right">[source]</a></h4>

> <code>ConvNav.last_frozen</code>(**`col`**=*`'child'`*)

Displays and returns the last frozen child container (`col='child'`), block container (`col='block'`) or layer (`col='layer'`).

In [ ]:
show_doc(ConvNav.first_unfrozen)

<h4 id="ConvNav.first_unfrozen" class="doc_header"><code>ConvNav.first_unfrozen</code><a href="__main__.py#L162" class="source_link" style="float:right">[source]</a></h4>

> <code>ConvNav.first_unfrozen</code>(**`col`**=*`'child'`*)

Displays and returns the first unfrozen child container (`col='child'`), block container (`col='block'`) or layer (`col='layer'`).

In [ ]:
show_doc(ConvNav.children)

<h4 id="ConvNav.children" class="doc_header"><code>ConvNav.children</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Display and return child container modules (equivelent to fastai `Learner.model.children()`)

In [ ]:
show_doc(ConvNav.blocks)

<h4 id="ConvNav.blocks" class="doc_header"><code>ConvNav.blocks</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Display and return block container modules

In [ ]:
show_doc(ConvNav.find_conv)

<h4 id="ConvNav.find_conv" class="doc_header"><code>ConvNav.find_conv</code><a href="__main__.py#L200" class="source_link" style="float:right">[source]</a></h4>

> <code>ConvNav.find_conv</code>(**`req`**=*`'all'`*, **`num`**=*`1`*, **`in_main`**=*`False`*)

Finds the first (`req=First`) or last (`req=last`) `num` Conv2d layers in the model body. Set `in_main=True` to return conv layers from the main body of the model only.

In [ ]:
show_doc(ConvNav.find_block)

<h4 id="ConvNav.find_block" class="doc_header"><code>ConvNav.find_block</code><a href="__main__.py#L225" class="source_link" style="float:right">[source]</a></h4>

> <code>ConvNav.find_block</code>(**`b`**, **`layers`**=*`True`*, **`layers_only`**=*`False`*)

Finds, displays and returns container blocks by module name `b`

Searchterm `b` should be the block's module name from `model.named_modules()`. Preceeding '0.' are removed so '0.0.6.1', '0.6.1' and '6.1' are all equivelent and match Container_block 1 of Container_child 6. Block layers are included by default. To return the block container element only, set `layers=False`. To just get the block's layers, set `layers_only=True`.

In [ ]:
show_doc(ConvNav.spread)

<h4 id="ConvNav.spread" class="doc_header"><code>ConvNav.spread</code><a href="__main__.py#L251" class="source_link" style="float:right">[source]</a></h4>

> <code>ConvNav.spread</code>(**`req`**=*`'conv'`*, **`num`**=*`5`*)

Returns `num` of equally spaced `req` elements over the model. `req` = 'conv', 'block', or 'child'

Spread is an ideal method for finding a small group of evenly spaced model elements (typically blocks) to use to investigate model activity, particularly for larger models (see `04_examples01.ipynb`). 

In [ ]:
%%capture
test_learner, test_summary, _ = get_test_vars()
cn_test = ConvNav(test_learner, test_summary)

test_eq(len(cn_test.search(0.6, exact=False, show=False)), 16)
cn_test.view()
cn_test.head
cn_test.body
cn_test.divs 
test_eq(len(cn_test.linear_layers), 2)                                  
test_eq(len(cn_test.dim_transitions), 5)
test_eq(len(cn_test.find_block('0.5.1')), 6)
test_eq(len(cn_test.find_block('0.5.1', layers=False)), 1)
test_eq(len(cn_test.find_conv('first', 5)), 5)
test_eq(len(cn_test.children), 8)
test_eq(len(cn_test.blocks), 8)
test_eq(len(cn_test.spread('conv', 8)), 7)
del(cn_test)

##Saving and loading CNDF dataframes.

In [ ]:
#export
def cndf_save(cn, filename, path='', with_modules=False):
  "Saves the instance dataframe `cn._cndf` to persistent storage at `path` with `filename` gzip compresseed"
  df = cn._cndf.copy()
  if not with_modules: df = df.iloc[:,:-1]
  with gzip.open(path+filename, "wb") as f:
    pickle.dump(df, f, pickle.HIGHEST_PROTOCOL)

In native format, CNDF dataframes include the module objects in a 'lyr_obj' column and the combined size of the module objects can be quite large, 100-200mb for a complex model such as a densenet or xresnet. Thus, by default, module objects are removed from the dataframe before saving. To save the model objects as well, check you have enough space in the download location and set `with_modules` to True. Dataframes are gzip compressed. 

In [ ]:
#export
def cndf_load(filename, path=''):
  "Loads a CNDF dataframe from persistent storage at `path`+`filename` and unzips it."
  with gzip.open(path+filename, "rb") as f:
    return pickle.load(f)

In [ ]:
#hide
# save example ConvNav Learner, Learner.summary() and and df to use in testing and exmaples

# with gzip.open("test_learner_resnet18", "wb") as f:
#     pickle.dump(learn, f, pickle.HIGHEST_PROTOCOL)

# with gzip.open("test_summary_resnet18", "wb") as f:
#     pickle.dump(learn.summary(), f, pickle.HIGHEST_PROTOCOL)
    
# df_test = cn_test._cndf.iloc[:,:-1]
# with gzip.open("test_df_resnet18", "wb") as f:
#     pickle.dump(df_test, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
#hide
test_df = None
test_learner = None
test_summary = None

In [ ]:
#hide
# download the example Convnav object and df
with gzip.open("test_df_resnet18", "rb") as f:
    test_df = pickle.load(f)

with gzip.open("test_summary_resnet18", "rb") as f:
    test_summary = pickle.load(f)

with gzip.open("test_learner_resnet18", "rb") as f:
    test_learner = pickle.load(f)